In [211]:
from pymystem3 import Mystem
from pprint import pprint
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
import json
import jsonlines
import collections
import string

### 2. Обрабатываем книгу с помощью mystem:

(шаманим, чтобы заработало)

In [22]:
m = Mystem(mystem_bin="C:/Users/Екатерина/Downloads/mystem-3.0-win7-32bit/mystem.exe")

In [25]:
with open('book.txt', encoding='utf-8') as f:
    text = f.read()

Открываем файл и (долго) лемматизируем:

In [26]:
lemmas = m.lemmatize(text)
lemmas[10:20]

[' ',
 'куролесова',
 '\n',
 '\n',
 '\n',
 '\n',
 '---------------------------------------------------------------\n',
 ' ',
 'по',
 ' ']

Записываем результат в файл:

In [196]:
with open('lemmas_mystem.txt', "w", encoding="utf-8") as f:
    f.write(''.join(lemmas))

Замеряем время работы (я взяла введение):

In [30]:
with open('book_intro.txt', encoding='utf-8') as f:
    text_intro = f.read()

In [31]:
%%time
analysis_intro = m.analyze(text_intro)

Wall time: 1min 52s


In [34]:
%%time
lemmas_intro = m.lemmatize(text_intro)

Wall time: 1min 46s


In [42]:
tokens_nltk = word_tokenize(text)
tokens_nltk[:10]

['Юрий', 'Иосифович', 'Коваль', '.', 'Приключения', 'Васи', 'Куролесова', '--', '--', '--']

### 3. Обрабатываем с помощью pymorphy:

Токенизируем через nltk:

In [44]:
tokens_intro = word_tokenize(text_intro)

In [43]:
morph = MorphAnalyzer()

Замеряем время работы pymorphy на отрывке:

In [47]:
%%time

analyses_intro = []

for token in tokens_intro:
    ana_intro = morph.parse(token)
    analyses_intro.append(ana_intro[0])

Wall time: 378 ms


Анализируем всю книгу pymorphy:

In [49]:
analyses = []

for token in tokens_nltk:
    ana = morph.parse(token)
    analyses.append(ana[0])

In [50]:
analyses[:2]

[Parse(word='юрий', tag=OpencorporaTag('NOUN,anim,masc,Name sing,nomn'), normal_form='юрий', score=1.0, methods_stack=((DictionaryAnalyzer(), 'юрий', 61, 0),)), Parse(word='иосифович', tag=OpencorporaTag('NOUN,anim,masc,Patr sing,nomn'), normal_form='иосифович', score=1.0, methods_stack=((DictionaryAnalyzer(), 'иосифович', 26, 0),))]

Записываем результаты анализа в файл:

In [74]:
with jsonlines.open('analysed_pymorphy.jsonl', mode='w') as analysed:
    for ana in analyses:
        new = {"lemma": ana.normal_form, "word": ana.word, "pos": ana.tag.POS}
        analysed.write(new)

## 4. Считаем и отвечаем на вопросы

In [181]:
with jsonlines.open('analysed_pymorphy.jsonl') as parsed:
    lines = []
    for i in parsed:
        lines.append(i)

In [182]:
lines[5]

{'lemma': 'вася', 'word': 'васи', 'pos': 'NOUN'}

Посчитаем долю вхождений каждой части речи:

In [183]:
all_pos = []
for line in lines:
    all_pos.append(line['pos'])

In [184]:
c = collections.Counter()

In [185]:
for pos in all_pos:
    c[pos] += 1

In [186]:
c

Counter({None: 5390, 'NOUN': 4147, 'VERB': 2580, 'PREP': 1523, 'CONJ': 1341, 'ADJF': 1210, 'NPRO': 962, 'ADVB': 880, 'PRCL': 782, 'INFN': 258, 'GRND': 180, 'ADJS': 89, 'PRED': 69, 'PRTF': 59, 'INTJ': 52, 'NUMR': 47, 'COMP': 45, 'PRTS': 25})

In [187]:
del c[None]

In [188]:
c

Counter({'NOUN': 4147, 'VERB': 2580, 'PREP': 1523, 'CONJ': 1341, 'ADJF': 1210, 'NPRO': 962, 'ADVB': 880, 'PRCL': 782, 'INFN': 258, 'GRND': 180, 'ADJS': 89, 'PRED': 69, 'PRTF': 59, 'INTJ': 52, 'NUMR': 47, 'COMP': 45, 'PRTS': 25})

In [189]:
pos_sum = sum(c.values()) 
pos_sum

14249

In [131]:
dict(c)

{'NOUN': 4147, 'PREP': 1523, 'NUMR': 47, 'PRTF': 59, 'ADJF': 1210, 'CONJ': 1341, 'NPRO': 962, 'VERB': 2580, 'PRCL': 782, 'ADVB': 880, 'INTJ': 52, 'GRND': 180, 'COMP': 45, 'PRTS': 25, 'ADJS': 89, 'INFN': 258, 'PRED': 69}

In [194]:
pos_freqs = []

for pos in c:
    pos_freq = (pos, round(c[pos] / pos_sum, 3))
    pos_freqs.append(pos_freq)

In [195]:
pos_freqs

[('NOUN', 0.291), ('PREP', 0.107), ('NUMR', 0.003), ('PRTF', 0.004), ('ADJF', 0.085), ('CONJ', 0.094), ('NPRO', 0.068), ('VERB', 0.181), ('PRCL', 0.055), ('ADVB', 0.062), ('INTJ', 0.004), ('GRND', 0.013), ('COMP', 0.003), ('PRTS', 0.002), ('ADJS', 0.006), ('INFN', 0.018), ('PRED', 0.005)]

Найдем 20 самых частотных глаголов:

In [172]:
verbs = []

for line in lines:
    if line['pos'] == 'VERB':
        verbs.append(line['lemma'])

In [173]:
v = collections.Counter()

In [174]:
for verb in verbs:
    v[verb] += 1

In [175]:
most_common_verbs = []

for i in v.most_common(20):#
    most_common_verbs.append(i[0])

In [176]:
most_common_verbs[1:]

['быть', 'думать', 'стать', 'сидеть', 'стоять', 'мочь', 'хотеть', 'крикнуть', 'пойти', 'спросить', 'глядеть', 'понять', 'ответить', 'говорить', 'знать', 'пахнуть', 'увидеть', 'взять', 'оказаться']

Аналогичным образом найдем 20 самых частотных наречий:

In [166]:
adverbs = []

for line in lines:
    if line['pos'] == 'ADVB':
        adverbs.append(line['lemma'])

In [167]:
adv = collections.Counter()

In [168]:
for adverb in adverbs:
    adv[adverb] += 1

In [169]:
most_common_adverbs = []

for i in adv.most_common(20):#
    most_common_adverbs.append(i[0])

In [171]:
most_common_adverbs

['только', 'тут', 'уже', 'теперь', 'совсем', 'потом', 'вдруг', 'где', 'уж', 'раз', 'там', 'снова', 'тогда', 'сразу', 'здесь', 'сейчас', 'домой', 'пока', 'зачем', 'ясно']

### Биграммы и триграммы

Чистим от пунктуации:

In [252]:
new_lemmas = []

for lemma in lemmas:
    if lemma.isalpha():
        new_lemmas.append(lemma)

In [253]:
new_lemmas

['юрий', 'иосифович', 'коваль', 'приключение', 'вася', 'куролесова', 'по', 'издание', 'пять', 'похищать', 'монах', 'м', 'детский', 'литература', 'перепечатка', 'и', 'spellcheck', 'м', 'филиппова', 'Spellcheck', 'Algis', 'что', 'я', 'нравиться', 'в', 'черный', 'лебедь', 'так', 'это', 'их', 'красный', 'нос', 'впрочем', 'к', 'наш', 'рассказ', 'это', 'не', 'иметь', 'никакой', 'отношение', 'хотя', 'в', 'тот', 'вечер', 'я', 'сидеть', 'на', 'лавочка', 'у', 'чистый', 'пруд', 'и', 'смотреть', 'как', 'раз', 'на', 'черный', 'лебедь', 'солнце', 'укатиться', 'за', 'почтамт', 'в', 'кинотеатр', 'колизей', 'грянуть', 'веселый', 'марш', 'и', 'тут', 'же', 'смениться', 'пулеметный', 'очередь', 'из', 'стеклянный', 'кафе', 'выходить', 'молодой', 'человек', 'и', 'распугивать', 'с', 'асфальт', 'сизарь', 'направляться', 'прямо', 'к', 'мой', 'скамейка', 'усаживаться', 'рядом', 'он', 'доставать', 'из', 'карман', 'часы', 'луковица', 'больше', 'похожий', 'на', 'репа', 'щелкнуть', 'крышка', 'и', 'в', 'тот', 'же', 

Собираем биграммы:

In [257]:
bigrms = list(nltk.bigrams(new_lemmas))
bigrms[:5]

[('юрий', 'иосифович'), ('иосифович', 'коваль'), ('коваль', 'приключение'), ('приключение', 'вася'), ('вася', 'куролесова')]

In [258]:
b = collections.Counter()

for bigrm in bigrms:
    b[bigrm] += 1

Вот топ-5 самых частотных. Первые две намекают, что в тексте много диалогов.

In [259]:
b.most_common(5)

[(('сказать', 'вася'), 36), (('сказать', 'болдырев'), 34), (('вася', 'и'), 28), (('в', 'мешок'), 27), (('у', 'он'), 24)]

In [261]:
trigrms = list(nltk.trigrams(new_lemmas))
trigrms[:5]

[('юрий', 'иосифович', 'коваль'), ('иосифович', 'коваль', 'приключение'), ('коваль', 'приключение', 'вася'), ('приключение', 'вася', 'куролесова'), ('вася', 'куролесова', 'по')]

А вот и триграммы:

In [262]:
t = collections.Counter()

for trigrm in trigrms:
    t[trigrm] += 1

In [263]:
t.most_common(5)

[(('и', 'тут', 'же'), 11), (('вася', 'и', 'болдырев'), 8), (('сказать', 'вася', 'и'), 6), (('он', 'доставать', 'из'), 5), (('в', 'деревня', 'сыч'), 5)]